In [ ]:
from json import loads
from requests import get
from time import sleep
from winotify import Notification

In [ ]:
ORIGINS = ['11320000'] # Origin City Code
DESTINATIONS = ['54450000', '54470000', '54310000', '54340000'] # Destination City Code
DATES = ['1401-12-28', '1401-12-29']
TRIES = 1000
SLEEP = 60 # in seconds

In [ ]:
t = 0
fullData = {}
notificationMessages = []

while t < TRIES:
    print(f'try number {t + 1}')
    for origin in ORIGINS:
        for destination in DESTINATIONS:
            for date in DATES:
                url = f'https://safar724.com/bus/getservices?origin={origin}&destination={destination}&date={date}'
                fullData[date] = loads(get(url).content)

                originEnglishName = fullData[date]["OriginEnglishName"]
                destinationEnglishName = fullData[date]["DestinationEnglishName"]

                for serviceData in fullData[date]['Items']:
                    availableSeatCount = serviceData['AvailableSeatCount']
                    serviceId = serviceData["ID"]

                    if availableSeatCount != 0:
                        notificationMessage = f'{originEnglishName} to {destinationEnglishName} in {date} with {availableSeatCount} available seats'
                        notificationUrl = f'https://safar724.com/bus/{originEnglishName}-{destinationEnglishName}?date={date}/#/service:{serviceId}-{destination}'

                        if notificationMessage not in notificationMessages:
                            notificationMessages.append(notificationMessage)
                            toast = Notification(app_id = 'Ticket Alert', title = 'New ticket', msg = notificationMessage)
                            toast.add_actions(label = 'Go to ticket', launch = notificationUrl)
                            toast.show()
                            print(notificationMessage, notificationUrl)
                        
                        sleep(1)
    t += 1
    sleep(SLEEP)